In [1]:
import os
os.chdir('/Users/furuku/Documents/thesis/2020 Thesis - Thailand Schdeule 2005-2019')
os.getcwd()

'/Users/furuku/Documents/thesis/2020 Thesis - Thailand Schdeule 2005-2019'

In [2]:
pwd

'/Users/furuku/Documents/thesis/2020 Thesis - Thailand Schdeule 2005-2019'

In [3]:
import pandas as pd

import json

In [4]:
from elasticsearch import Elasticsearch 
from elasticsearch.helpers import streaming_bulk
from elasticsearch.helpers import scan

es=Elasticsearch([{'host':'localhost','port':9200}])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [5]:
es.ping()

False

In [11]:
def create_index(index: str):
    es.indices.create(
        index=index,
        body={
            "settings": {"number_of_shards": 1},
            "mappings": {
                "properties": {
                    "Time": {"type": "date"},
                    "Alliance": {"type": "keyword"},
                    "Op Al": {"type": "keyword"},
                    "Orig": {"type": "keyword"},
                    "Dest": {"type": "keyword"},
                    "Kilometers": {"type": "float"},
                    "Flight": {"type": "integer"},
                    "Sequence": {"type": "integer"},
                    "Equip": {"type": "keyword"},
                    "Seats": {"type": "integer"},
                    "Dep Time": {"type": "integer"},
                    "Arr Time": {"type": "integer"},
                    "Block Mins": {"type": "integer"},
                    "Region_orig": {"type": "keyword"},
                    "country_orig": {"type": "keyword"},
                    "Region_dest": {"type": "keyword"},
                    "country_dest": {"type": "keyword"},

                    
                }
            },
        },
        ignore=400,
    )
    


In [12]:
def gendata(table,inp):
    
    for row in table:
        
        yield {
            
            "_index" : inp,
#             '_type' : '_doc',
            '_source' : row
            
        }

In [13]:
def make_data(y2019,country,thai):
    data_airport = country.append(thai)
    region = data_airport[['Code', 'Global Region','Country Name']]
    # region = region.rename(columns={"IATA Metro": "Orig"})
#     print(region.head(10))
    region = region.drop_duplicates()
    region = region.set_index('Code')
    region = region.dropna()
    
    y2019 = y2019[(y2019.Dest == "BKK") | (y2019.Orig == "BKK")]
    
    analyze = y2019.join(region, on=['Orig'])
    analyze = analyze.rename(columns={"Global Region": "Region_orig", "Country Name": "country_orig"})
    analyze = analyze.join(region, on=['Dest'])
    analyze = analyze.rename(columns={"Global Region": "Region_dest", "Country Name": "country_dest"})

    lis = []
    
    for index,row in analyze.iterrows():
        if(row['Orig'] == 'BKK'):
            tmp = str(row['Dep Time'])
        else:
            tmp = str(row['Arr Time'])
        if len(tmp) < 4 :
            add = 4 - len(tmp)
            tmp = ("0" * add) + tmp
        if tmp == '2400':
            tmp = '2359'

        st = str(row['Date']) +' '+ tmp[:-2] + ':' +  tmp[-2:] + ':00' 
        lis.append(st)
        
    analyze['Time'] = lis
    analyze['Time'] = pd.to_datetime(analyze['Time'] )
    
    
#     arrive.remove('BKK')
#     test = pd.DataFrame(arrive)
#     test = test.join(region, on=0).rename(columns = {0:'station'})
    
    
    return analyze
    

In [14]:
def preprocess_connect(data):
    data = data[['Time', 'Alliance', 'Op Al', 'Orig', 'Dest', 'Kilometers',
       'Flight', 'Sequence', 'Equip', 'Seats',
       'Dep Time', 'Arr Time', 'Block Mins','Region_orig',	'country_orig',	'Region_dest',	'country_dest']]
    
    data = data[(data['Time'].map(type)  != str) &  (data['Time'].notnull()) ]
    data = data.dropna(subset=['Orig','Dest'])
    data['Seats'] = data['Seats'].astype(int)
    
    if (data['Time'].dtypes != ('O')):
        data['Time'] = data['Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    else:
        data['Time'] = pd.to_datetime(data['Travel Month'], format='%Y-%m-%d %H:%M:%S')
        data['Time'] = data['Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    
    return data

In [15]:
bank_y2005 = pd.read_excel("./2005_Schedule_Daily_Extract_Report_1884.xlsx") 
bank_y2006 = pd.read_excel("./2006_Schedule_Daily_Extract_Report_3444.xlsx") 
bank_y2007 = pd.read_excel("./2007_Schedule_Daily_Extract_Report_6156.xlsx") 
bank_y2008 = pd.read_excel("./2008_Schedule_Daily_Extract_Report_6225.xlsx") 
bank_y2009 = pd.read_excel("./2009_Schedule_Daily_Extract_Report_6320.xlsx") 
bank_y2010 = pd.read_excel("./2010_Schedule_Daily_Extract_Report_6412.xlsx") 
bank_y2011 = pd.read_excel("./2011_Schedule_Daily_Extract_Report_6498.xlsx") 
bank_y2012 = pd.read_excel("./2012_Schedule_Daily_Extract_Report_6568.xlsx") 
bank_y2013 = pd.read_excel("./2013_Schedule_Daily_Extract_Report_6775.xlsx") 
bank_y2014 = pd.read_excel("./2014_Schedule_Daily_Extract_Report_7224.xlsx") 
bank_y2015 = pd.read_excel("./2015_Schedule_Daily_Extract_Report_7551.xlsx") 
bank_y2016 = pd.read_excel("./2016_Schedule_Daily_Extract_Report_7640.xlsx") 
bank_y2017 = pd.read_excel("./2017_Schedule_Daily_Extract_Report_7758.xlsx") 
bank_y2018 = pd.read_excel("./2018_Schedule_Daily_Extract_Report_7912.xlsx") 
bank_y2019 = pd.read_excel("./2019_Schedule_Daily_Extract_Report_8030.xlsx") 

In [16]:
# y2005_07_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2005-2007 BKK as STOP.xlsx")  
# y2008_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2008 BKK as STOP.xlsx") 
# y2009_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2009 BKK as STOP.xlsx") 
# y2010_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2010 BKK as STOP.xlsx") 
# y2011_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2011 BKK as STOP.xlsx") 
# y2012_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2012 BKK as STOP.xlsx") 
# y2013_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2013 BKK as STOP.xlsx") 
# y2014_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2014 BKK as STOP.xlsx") 
# y2015_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2015 BKK as STOP.xlsx") 
# y2016_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2016 BKK as STOP.xlsx") 
# y2017_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2017 BKK as STOP.xlsx") 
# y2018_19_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_Jan2018-Dec2019 BKK as STOP.xlsx") 

In [17]:

country = pd.read_excel("../2020 Thesis - Thailand Schdeule 2005-2019/1.-Airport_Lookup_Country-Region.xlsx")
thai = pd.read_excel("../2020 Thesis - Thailand Schdeule 2005-2019/Thailand_Airport_Lookup.xlsx")


In [18]:
data = make_data(bank_y2019,country,thai)


In [19]:
data.columns

Index(['Date', 'Mkt Al', 'Alliance', 'Op Al', 'Orig', 'Dest', 'Kilometers',
       'Flight', 'Sequence', 'Routing', 'Stops', 'Equip', 'Seats', 'First',
       'Business', 'Prem Econ', 'Econ', 'Other', 'Dep Term', 'Arr Term',
       'Dep Time', 'Arr Time', 'Block Mins', 'Arr Flag', 'Orig WAC',
       'Dest WAC', 'Region_orig', 'country_orig', 'Region_dest',
       'country_dest', 'Time'],
      dtype='object')

In [20]:
data

,Date,Mkt Al,Alliance,Op Al,Orig,Dest,Kilometers,Flight,Sequence,Routing,...,Arr Time,Block Mins,Arr Flag,Orig WAC,Dest WAC,Region_orig,country_orig,Region_dest,country_dest,Time
0,2019-01-01,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,512,1,BKK-SIN,...,1255,155,0,782,776,Asia,Thailand,Asia,Singapore,2019-01-01 09:20:00
1,2019-01-01,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,520,1,BKK-SIN,...,1525,155,0,782,776,Asia,Thailand,Asia,Singapore,2019-01-01 11:50:00
2,2019-01-01,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,516,1,BKK-SIN,...,1705,150,0,782,776,Asia,Thailand,Asia,Singapore,2019-01-01 13:35:00
3,2019-01-01,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,514,1,BKK-SIN,...,50,150,1,782,776,Asia,Thailand,Asia,Singapore,2019-01-01 21:20:00
7,2019-01-01,3K,Low Cost Carriers,3K,SIN,BKK,1411.394693,511,1,SIN-BKK,...,840,150,0,776,782,Asia,Singapore,Asia,Thailand,2019-01-01 08:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772835,2019-12-31,ZH,Star Alliance,ZH,BKK,SZX,1699.467270,9004,1,BKK-SZX,...,1915,190,0,782,713,Asia,Thailand,Asia,China,2019-12-31 15:05:00
772836,2019-12-31,ZH,Star Alliance,ZH,BKK,SZX,1699.467270,9002,1,BKK-SZX,...,2335,185,0,782,713,Asia,Thailand,Asia,China,2019-12-31 19:30:00
772838,2019-12-31,ZH,Star Alliance,ZH,SZX,BKK,1699.467270,9003,1,SZX-BKK,...,1405,190,0,713,782,Asia,China,Asia,Thailand,2019-12-31 14:05:00
772839,2019-12-31,ZH,Star Alliance,ZH,SZX,BKK,1699.467270,9001,1,SZX-BKK,...,1830,190,0,713,782,Asia,China,Asia,Thailand,2019-12-31 18:30:00


In [21]:

data = preprocess_connect(data)
data.dtypes

Time             object
Alliance         object
Op Al            object
Orig             object
Dest             object
Kilometers      float64
Flight            int64
Sequence          int64
Equip            object
Seats             int64
Dep Time          int64
Arr Time          int64
Block Mins        int64
Region_orig      object
country_orig     object
Region_dest      object
country_dest     object
dtype: object

In [22]:
# make_data(bank_y2019,country,thai)

In [23]:
bank_y2019

,Date,Mkt Al,Alliance,Op Al,Orig,Dest,Kilometers,Flight,Sequence,Routing,...,Econ,Other,Dep Term,Arr Term,Dep Time,Arr Time,Block Mins,Arr Flag,Orig WAC,Dest WAC
0,2019-01-01,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,512,1,BKK-SIN,...,0,0,NaN,1,920,1255,155,0,782,776
1,2019-01-01,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,520,1,BKK-SIN,...,0,0,NaN,1,1150,1525,155,0,782,776
2,2019-01-01,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,516,1,BKK-SIN,...,0,0,NaN,1,1335,1705,150,0,782,776
3,2019-01-01,3K,Low Cost Carriers,3K,BKK,SIN,1411.394693,514,1,BKK-SIN,...,0,0,NaN,1,2120,50,150,1,782,776
4,2019-01-01,3K,Low Cost Carriers,3K,HKT,SIN,976.871811,534,1,HKT-SIN,...,0,0,D,1,935,1230,115,0,782,776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772837,2019-12-31,ZH,Star Alliance,ZH,HKT,SZX,2311.017992,9016,1,HKT-SZX,...,159,0,I,NaN,10,500,230,0,782,713
772838,2019-12-31,ZH,Star Alliance,ZH,SZX,BKK,1699.467270,9003,1,SZX-BKK,...,153,0,NaN,NaN,1155,1405,190,0,713,782
772839,2019-12-31,ZH,Star Alliance,ZH,SZX,BKK,1699.467270,9001,1,SZX-BKK,...,159,0,NaN,NaN,1620,1830,190,0,713,782
772840,2019-12-31,ZH,Star Alliance,ZH,SZX,BKK,1699.467270,9019,1,SZX-BKK,...,153,0,NaN,NaN,2245,125,220,1,713,782


In [24]:
add_elastic = make_data(bank_y2019,country,thai)
add_elastic = preprocess_connect(add_elastic)
add_elastic

,Time,Alliance,Op Al,Orig,Dest,Kilometers,Flight,Sequence,Equip,Seats,Dep Time,Arr Time,Block Mins,Region_orig,country_orig,Region_dest,country_dest
0,2019-01-01 09:20:00,Low Cost Carriers,3K,BKK,SIN,1411.394693,512,1,320,180,920,1255,155,Asia,Thailand,Asia,Singapore
1,2019-01-01 11:50:00,Low Cost Carriers,3K,BKK,SIN,1411.394693,520,1,320,180,1150,1525,155,Asia,Thailand,Asia,Singapore
2,2019-01-01 13:35:00,Low Cost Carriers,3K,BKK,SIN,1411.394693,516,1,320,180,1335,1705,150,Asia,Thailand,Asia,Singapore
3,2019-01-01 21:20:00,Low Cost Carriers,3K,BKK,SIN,1411.394693,514,1,320,180,2120,50,150,Asia,Thailand,Asia,Singapore
7,2019-01-01 08:40:00,Low Cost Carriers,3K,SIN,BKK,1411.394693,511,1,320,180,710,840,150,Asia,Singapore,Asia,Thailand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772835,2019-12-31 15:05:00,Star Alliance,ZH,BKK,SZX,1699.467270,9004,1,738,161,1505,1915,190,Asia,Thailand,Asia,China
772836,2019-12-31 19:30:00,Star Alliance,ZH,BKK,SZX,1699.467270,9002,1,738,167,1930,2335,185,Asia,Thailand,Asia,China
772838,2019-12-31 14:05:00,Star Alliance,ZH,SZX,BKK,1699.467270,9003,1,738,161,1155,1405,190,Asia,China,Asia,Thailand
772839,2019-12-31 18:30:00,Star Alliance,ZH,SZX,BKK,1699.467270,9001,1,738,167,1620,1830,190,Asia,China,Asia,Thailand


In [25]:
index = ['bank_y2005','bank_y2006','bank_y2007','bank_y2008','bank_y2009','bank_y2010','bank_y2011',
         'bank_y2012','bank_y2013','bank_y2014','bank_y2015','bank_y2016','bank_y2017','bank_y2018',
         'bank_y2019']

table = [bank_y2005, bank_y2006, bank_y2007, bank_y2008, bank_y2009, bank_y2010, bank_y2011,
         bank_y2012, bank_y2013, bank_y2014, bank_y2015, bank_y2016, bank_y2017, bank_y2018,
         bank_y2019]

for idx,name in enumerate(table):
    
    print("Process in " + index[idx] )
    
    tmp = name.copy()
    add_elastic = make_data(tmp,country,thai)
    add_elastic = preprocess_connect(add_elastic)
    
    json_str = add_elastic.to_json(orient='records')
    json_records = json.loads(json_str) 
    
    inserted = 0


    for success, info in streaming_bulk(
        client=es, actions=gendata(json_records,index[idx]),request_timeout=50
        ):
        inserted += success
        
#     print(inserted)
    
    if add_elastic.shape[0] == inserted :
        print(str(index[idx]) + " SUCCESS!!")
        print()
    else:
        print(str(index[idx]) + " ERROR!!")
        print(add_elastic.shape[0],inserted)
        print()
    
    
    
    



# tmp = data_stop.copy()
# add_elastic = preprocess_connect(tmp)

Process in bank_y2005
bank_y2005 SUCCESS!!

Process in bank_y2006
bank_y2006 SUCCESS!!

Process in bank_y2007
bank_y2007 SUCCESS!!

Process in bank_y2008
bank_y2008 SUCCESS!!

Process in bank_y2009
bank_y2009 SUCCESS!!

Process in bank_y2010
bank_y2010 SUCCESS!!

Process in bank_y2011
bank_y2011 SUCCESS!!

Process in bank_y2012
bank_y2012 SUCCESS!!

Process in bank_y2013
bank_y2013 SUCCESS!!

Process in bank_y2014
bank_y2014 SUCCESS!!

Process in bank_y2015
bank_y2015 SUCCESS!!

Process in bank_y2016
bank_y2016 SUCCESS!!

Process in bank_y2017
bank_y2017 SUCCESS!!

Process in bank_y2018
bank_y2018 SUCCESS!!

Process in bank_y2019
bank_y2019 SUCCESS!!



In [26]:
tttt = y2008_stop.copy()
tttt = tttt[(tttt['Travel Month'].map(type)  != str) &  (tttt['Travel Month'].notnull()) ]
tttt



NameError: name 'y2008_stop' is not defined

In [ ]:
y2008_stop.columns

In [ ]:
# y2008_stop
# y2008_stop['Travel Month'].dt.strftime('%Y-%m-%d')
y2008_stop[~(y2008_stop['Travel Month'].isnull())].loc[131312]['Travel Month']

In [ ]:
for j in list(y2008_stop['Travel Month'].unique()):
    print(type(j))

In [ ]:
# y2008_stop.select_dtypes('datetime')
y2008_stop[(y2008_stop['Travel Month'].map(type)  != str) &  (y2008_stop['Travel Month'].notnull()) ]

In [ ]:
ttt = preprocess_connect(y2019_stop.copy())
ttt

In [ ]:
json_str = ttt.to_json(orient='records')
json_records = json.loads(json_str)

In [ ]:
for i,j in enumerate(json_records):
    if(i == 20):
        break;
    print(j)

In [ ]:
# add_data = gendata(json_records,'test')

In [ ]:
# try:
#     helpers.bulk(es, gendata(json_records,'test'))
#     print('ok')
# except Exception as e:
#     print('on')
    

In [ ]:
inserted = 0


for success, info in streaming_bulk(
    client=es, actions=gendata(json_records,'test'),request_timeout=50
    ):
    inserted += success

In [ ]:
inserted

In [ ]:
index = ['y2005','y2006','y2007','y2008','y2009','y2010','y2011',
         'y2012','y2013','y2014','y2015','y2016','y2017','y2018',
         'y2019','y2005_stop','y2006_stop','y2007_stop','y2008_stop','y2009_stop','y2010_stop','y2011_stop',
         'y2012_stop','y2013_stop','y2014_stop','y2015_stop','y2016_stop','y2017_stop','y2018_stop','y2019_stop',]



In [ ]:
if __name__ == '__main__':

for name in index:    
        try:
            # create index and index must be lowercase alphabet
            create_index(name.lower())
            print(index,"created successed")
        except Exception as e:
            print(e)
            print(index,"create not success")

In [ ]:
query_body = {
  "query": {
    "match": {
      "Orig": "BKK"
    }
  }
}





# {
#   "query": {
#     "bool": {
#       "filter": {
#         "term": {
#           "Stop2": "BKK"
#         }
#       }
#     }
#   }
# }








In [ ]:
results_gen = scan(
    es,
    query=query_body,
    index='y2019',
)

results = list(results_gen)

In [ ]:
# json.dumps([i['_source'] for i in results ])

In [ ]:
res = [i['_source'] for i in results ]

# es.search(index="test", body=query_body)['hits']['hits']

In [ ]:
res

In [ ]:
dd = pd.DataFrame(res)
dd

In [ ]:
dd[~dd.Stop2.isna()]

In [ ]:
tmp[tmp.Stop1 == 'BKK'].shape

In [ ]:
results

In [ ]:

tttt = preprocess_connect(y2008_stop)
tttt

In [ ]:
y2017_stop

In [ ]:
data = pd.read_excel("./y2017_foreign.xlsx")
country = pd.read_excel("../2020 Thesis - Thailand Schdeule 2005-2019/1.-Airport_Lookup_Country-Region.xlsx")
thai = pd.read_excel("../2020 Thesis - Thailand Schdeule 2005-2019/Thailand_Airport_Lookup.xlsx")